# Miniproject 1
Duke University: ECE 580
Francisco Reveriano

In [1]:
# Import Necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
from DCT.DCT_Matrix import *
from Regression_compressive_sensing import imgRead, imgShow, imgRecover

Original Image
X: 200 Y: 192
(64, 64)


## Load Images

We First Proceed To Load the Boat Image. We then check the dimensions to make sure that the dimensions are single channel. We can then proceed to process the image. 

In [2]:
# Read/Load Images
boat = 'fishing_boat.bmp'
lena = "lena.bmp"

## Read the Boat Image
matrix = imgRead(boat)
P, Q = matrix.shape
print("Boat Image Dimensions")
print("X:", P, "Y",Q)

Boat Image Dimensions
X: 200 Y 192


## Prepare the DCT(T) Matrix

To calculate the DCT Matrix we proceed to use the formula given in the powerpoint. 

In [3]:
# We First Declare the Dimensions for the T Matrix 
dimension = (8,8)
## We then call the DCT_Matrix function to create the DCT Matrix
T_Matrix = DCT_Matrix(dimension[0], dimension[1])
### We then Check that the T_Matrix is the correct dimensions
print("Transformation Shape")
print(T_Matrix.shape)

Transformation Shape
(64, 64)
